In [5]:
import pandas as pd
import string
import requests
from datetime import datetime, timedelta
from sqlalchemy import create_engine

# Json com a lista das palavras chave, e as categorias informadas no documento da Pfizer
lista_de_palavras = {'palavras_chave':
                     ['COVID AND (vacina OR antiviral)',
                         'COVID-19 AND (vacina OR antiviral)',
                         'Coronavirus AND (vacina OR antiviral)',
                         'antiviral AND (covid OR covid-19)',
                         'nirmatrelvir',
                         'ritonavir',
                         'Paxlovid',
                         'COVID-19 AND treatment',
                         'COVID AND treatment',    
                         '"bivalent vaccine" AND(covid OR covid-19)',
                         'covid AND test',
                         'Covid-19 AND test',
                         '"vaccination campaign" AND(covid OR covid-19)',
                         'vaccination AND (covid OR covid-19)',
                         'Comirnaty',
                         '"attenuated vaccine" AND (covid OR covid-19)',
                         '"inactivated vaccine" AND (covid OR covid-19)',
                         '"Symptoms of Coronavirus"',
                         '"Symptoms of Covid-19"',
                         '"Symptoms of Covid"',
                         '"Minister of Health" AND (covid OR covid-19)',
                         'SBIm AND (covid OR covid-19)',
                         '"Immunization journey" AND (covid OR covid-19)',
                         '"vaccine efficacy" AND (covid OR covid-19)',
                         '"vaccine security" AND (covid OR covid-19)',
                         '"RNA vaccine" AND(Comirnaty OR RNA mensageiro)',
                         'Coronavirus AND (covid OR covid-19)',
                         'Hospitalization AND (covid OR covid-19)',
                         'Imunization AND (covid OR covid-19)',
                         'Pandemic AND (covid OR covid-19)',
                         'sequels AND (covid OR covid-19)',
                         '"long covid " AND (covid OR covid-19)',
                         'immunosuppressed AND (covid OR covid-19)',
                         'immunocompromised AND (covid OR covid-19)',
                         'diagnosis AND (covid OR covid-19)'],
                     'categorias': {
                         'Cardiologia':
                         ['CVA',
                          'stroke',
                          'cardiac arrythmia',
                          'Atrial Fibrillation',
                          'Thrombosis',
                          'DVT',
                          'VTE',
                          'Pulmonary Embolism',
                          'Heart Failure',
                          'High blood pressure',
                          'Hypertension',
                          ],
                         'Hematologia':
                         ['Acute Leukemia',
                          'Leukemia',
                          'Myeloid',
                          'Lymphoid',
                          'Multiple Myeloma',
                          'Acute Lymphoblastic Leukemia',
                          'Myeloid',
                          'Acute Lymphoblastic Leukemia',
                          'Acute Myeloid Leukemia',
                          'hemophilia',
                          ],
                         'SNC': ['Depression',
                                 'Anxiety',
                                 'Antidepressant',
                                 'Panic Syndrome',
                                 'MDD',
                                 'Major Depressive Disorder',
                                 'Depressed',
                                 'Anxious',
                                 'Mental Health',
                                 'Mental Disease',
                                 'Mental disorder',
                                 'Sadness',
                                 'Suicide',
                                 'Obesity',
                                 'Burnout',
                                 'TAG',

                                 ],
                         'Oncologia': [
                             'Cancer',
                             'Carcinoma',
                             'Neoplasm',
                             'Tumor',
                             'Immunotherapy',
                             'Chemotherapy',
                             'Hormone therapy',
                             'Targeted therapy',
                             'Transplant',
                             'bone marrow',
                             'Kinase',
                             'Overall survival',
                             'Monotherapy',
                             'Breast Cancer',
                             'Kidney cancer',
                             'Bladder Cancer',
                             'Kidney neoplasm',
                             'Gastrointestinal tract',
                             'Pancreatic neuroendocrine tumors',
                             'Lung Cancer',
                             'Lung Tumor',
                             'Skin cancer',
                             'Melanoma',
                             'Colorectal',
                             'Pierre Fabre',
                             'Skin Carcinoma',
                         ],
                         'Reumatologia': [
                             'Anti-drug antibody',
                             'Anti-Neutralizing Drug Antibody',
                             'Monoclonal Antibody',
                             'Arthritis',
                             'Juvenile Idiopathic Arthritis',
                             'Polyarticular Juvenile Idiopathic Arthritis',
                             'psoriatic arthritis',
                             'psoriatic arthritis',
                             'Enthesitis Related Arthritis',
                             'Rheumatoid arthritis',
                             'Biological',
                             'Brazilian Congress of Rheumatology',
                             'Immune-mediated diseases',
                             'Inflammatory Diseases',
                             'joint pain',
                             'Ankylosing spondylitis',
                             'Non-Radiographic Axial Spondyloarthritis',
                             'tumor necrosis factor',
                             'Herpes Zoster',
                             'Backache',
                             'tumor necrosis',
                             'Brazilian Society of Rheumatology',
                             'biological therapy',
                             'Thrombosis',
                             'Tuberculosis'


                         ]

                     }}


# função de busca na API
def apisearch(query: string):
    # Definir os parâmetros de busca
    search_params = {        
        "q": query,
        "lang": "en",
        "sort_by": "relevancy",
        "page_size": 100
    }

    # Obter as notícias nos últimos 24 horas
    end_date = datetime.utcnow()
    start_date = end_date - timedelta(days=1)
    search_params["from"] = start_date.strftime("%Y-%m-%d")
    search_params["to"] = end_date.strftime("%Y-%m-%d")

    # Adiciona a chave de API da Newscatcher
    # api_key = "BbJk1vWYhB6jK6BJAoR8NFXL-acL8pqF0jdIQbbxI9A>"
    headers = {"x-api-key": "BbJk1vWYhB6jK6BJAoR8NFXL-acL8pqF0jdIQbbxI9A"}

    # Faz a requisição para a  API
    url = "https://api.newscatcherapi.com/v2/search"
    response = requests.get(url, headers=headers, params=search_params)

    return response

#Salva o dataframe na base de dados
def saveInDatabase(dataframe):
    from sqlalchemy import create_engine

    host1 = 'pfizerdb.cqxct8aqwlso.us-east-2.rds.amazonaws.com'
    port1 = '3306'
    user1 = 'pfizer'
    passw1 = 'P!zer2597!'
    database1 = 'pfizerdb'

    mydb1 = create_engine('mysql+pymysql://' + user1 + ':' + passw1 +
                          '@' + host1 + ':' + str(port1) + '/' + database1, echo=False)
    dataframe.to_sql(name='news_covid_mundo', con=mydb1,
                     if_exists='append', index=False)


In [6]:
#Ao final do processo esse dataframe estará com todos dos dados das pesquisas concatenados
df_final = pd.DataFrame()

#Neste bloco, para cada palavra-chave do json, é feito um loop nas categorias
#             e nas áreas de interesse
for i in lista_de_palavras['palavras_chave']:
    print('*** Palavra-chave:'+ i)
    for c in lista_de_palavras['categorias']:
        print('** Categoria: '+ c)
        for k in lista_de_palavras['categorias'][c]:
            str_de_busca = i+' ' + ' "' + k + '"'
            print(str_de_busca) 
            response = apisearch(str_de_busca)
            
            try:
             articles = response.json()["articles"]
             df = pd.DataFrame(articles, columns=[
                "_id", "title", "published_date", "link"])
             df['category'] = 'COVID'
             df['associated'] = c
             df['keyword'] = k

        # Imprimir o dataframe
             df_final = pd.concat([df_final, df], ignore_index=True)
            except:
             continue
#refatora o json, agrupando as notícias duplicadas e adicionando os campos 'keyword' em um único campo, separados por vírgula
df_refatorado = df_final.groupby(["_id", "title", "published_date", "link", "category", "associated"])['keyword'].agg(
    lambda x: ', '.join(x)).reset_index()

#renomeia as colunas
df_refatorado = df_refatorado.rename(
         columns={'published_date': 'published', '_id': 'id', 'keyword':'kw','category':'categoria'})
#df_refatorado.to_excel('teste.xlsx')

saveInDatabase(df_refatorado)

*** Palavra-chave:COVID AND (vacina OR antiviral)
** Categoria: Cardiologia
COVID AND (vacina OR antiviral)  "CVA"
COVID AND (vacina OR antiviral)  "stroke"
COVID AND (vacina OR antiviral)  "cardiac arrythmia"
COVID AND (vacina OR antiviral)  "Atrial Fibrillation"
COVID AND (vacina OR antiviral)  "Thrombosis"
COVID AND (vacina OR antiviral)  "DVT"
COVID AND (vacina OR antiviral)  "VTE"
COVID AND (vacina OR antiviral)  "Pulmonary Embolism"
COVID AND (vacina OR antiviral)  "Heart Failure"
COVID AND (vacina OR antiviral)  "High blood pressure"
COVID AND (vacina OR antiviral)  "Hypertension"
** Categoria: Hematologia
COVID AND (vacina OR antiviral)  "Acute Leukemia"
COVID AND (vacina OR antiviral)  "Leukemia"
COVID AND (vacina OR antiviral)  "Myeloid"
COVID AND (vacina OR antiviral)  "Lymphoid"
COVID AND (vacina OR antiviral)  "Multiple Myeloma"
COVID AND (vacina OR antiviral)  "Acute Lymphoblastic Leukemia"
COVID AND (vacina OR antiviral)  "Myeloid"
COVID AND (vacina OR antiviral)  "Acute